In [141]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [142]:
df = pd.read_csv('jugadores_fantasy_ultrainstinto.csv')

In [143]:
original_data = df[['Nombre', 'posicion']].copy()

In [144]:
df = pd.get_dummies(df, columns=['posicion', 'equipo'])


In [145]:
X = df.drop(['Nombre', 'Total_puntos', 'Precio'], axis=1)
y_puntos = df['Total_puntos']
y_precio = df['Precio']

In [146]:
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X, y_puntos, test_size=0.3, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X, y_precio, test_size=0.3, random_state=42)


In [147]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [148]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [149]:
rf_puntos = RandomForestRegressor(random_state=42)
grid_search_puntos = GridSearchCV(estimator=rf_puntos, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search_puntos.fit(X_train_scaled, y_puntos_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s

[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END m

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [150]:
best_rf_puntos = grid_search_puntos.best_estimator_

In [151]:
cv_scores_puntos = cross_val_score(best_rf_puntos, X_train_scaled, y_puntos_train, cv=5)
print("Puntos CV Scores:", cv_scores_puntos)
print("Puntos CV Mean Score:", cv_scores_puntos.mean())

Puntos CV Scores: [0.81589768 0.78711989 0.79658953 0.74472669 0.78924303]
Puntos CV Mean Score: 0.7867153634624765


In [152]:
rf_precio = RandomForestRegressor(random_state=42)
grid_search_precio = GridSearchCV(estimator=rf_precio, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search_precio.fit(X_train_scaled, y_precio_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END m

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [153]:
best_rf_precio = grid_search_precio.best_estimator_

In [154]:
cv_scores_precio = cross_val_score(best_rf_precio, X_train_scaled, y_precio_train, cv=5)
print("Precio CV Scores:", cv_scores_precio)
print("Precio CV Mean Score:", cv_scores_precio.mean())

Precio CV Scores: [0.64224173 0.72259554 0.62994221 0.56320689 0.61272223]
Precio CV Mean Score: 0.6341417201744849


In [155]:
y_puntos_pred = best_rf_puntos.predict(X_test_scaled)
y_precio_pred = best_rf_precio.predict(X_test_scaled)

In [156]:
print("Puntos R2 Score:", best_rf_puntos.score(X_test_scaled, y_puntos_test))
print("Puntos MSE:", mean_squared_error(y_puntos_test, y_puntos_pred))
print("Precio R2 Score:", best_rf_precio.score(X_test_scaled, y_precio_test))
print("Precio MSE:", mean_squared_error(y_precio_test, y_precio_pred))

Puntos R2 Score: 0.8731140266530777
Puntos MSE: 169.47623006439218
Precio R2 Score: 0.7547395336270623
Precio MSE: 9374937480822.928


In [157]:
resultados = pd.DataFrame({
    'Nombre': original_data.loc[X_test.index, 'Nombre'],
    'Posicion': original_data.loc[X_test.index, 'posicion'],
    'Puntos_Predichos': y_puntos_pred,
    'Precio_Predicho': y_precio_pred
})

In [158]:
top_10_puntos = resultados.sort_values(by='Puntos_Predichos', ascending=False).head(10)
print("\n10 jugadores con mayor puntaje predicho:")
for i, row in top_10_puntos.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Precio: {row['Precio_Predicho']:.2f} €")

top_10_puntos = top_10_puntos.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_puntos.to_csv('sugerencia.csv', index=False)

print("Datos exportados a sugerencia.csv")


10 jugadores con mayor puntaje predicho:
A. Griezmann - Puntos: 129.14, Precio: 17211813.34 €
J. Bellingham - Puntos: 127.72, Precio: 17147016.67 €
N. Williams - Puntos: 122.38, Precio: 17343248.22 €
A. Budimir - Puntos: 119.98, Precio: 16501073.34 €
V. Muriqi - Puntos: 113.52, Precio: 16506528.34 €
Sávio - Puntos: 93.72, Precio: 15633492.32 €
R. Lewandowski - Puntos: 90.33, Precio: 10530381.41 €
B. Zaragoza - Puntos: 89.75, Precio: 15381661.97 €
G. Moreno - Puntos: 89.41, Precio: 12564471.07 €
J. Cancelo - Puntos: 87.35, Precio: 9916051.15 €
Datos exportados a sugerencia.csv


In [159]:
top_10_precio = resultados.sort_values(by='Precio_Predicho', ascending=False).head(10)
print("\n10 jugadores con mayor precio predicho:")
for i, row in top_10_precio.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Precio: {row['Precio_Predicho']:.2f} €")


10 jugadores con mayor precio predicho:
N. Williams - Puntos: 122.38, Precio: 17343248.22 €
A. Griezmann - Puntos: 129.14, Precio: 17211813.34 €
J. Bellingham - Puntos: 127.72, Precio: 17147016.67 €
V. Muriqi - Puntos: 113.52, Precio: 16506528.34 €
A. Budimir - Puntos: 119.98, Precio: 16501073.34 €
Sávio - Puntos: 93.72, Precio: 15633492.32 €
B. Zaragoza - Puntos: 89.75, Precio: 15381661.97 €
G. Moreno - Puntos: 89.41, Precio: 12564471.07 €
B. Díaz - Puntos: 73.82, Precio: 12289432.84 €
T. Kroos - Puntos: 84.40, Precio: 11753885.66 €


In [160]:
top_10_delanteros = resultados[resultados['Posicion'] == 'delantero'].sort_values(by='Puntos_Predichos', ascending=False).head(10)

print("\n10 delanteros con mayor puntaje predicho:")
for i, row in top_10_delanteros.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Posición: {row['Posicion']}, Precio: {row['Precio_Predicho']:.2f} €")

top_10_delanteros = top_10_delanteros.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_delanteros.to_csv('sugerencia_delanteros.csv', index=False)
print("Datos de delanteros exportados a sugerencia_delanteros.csv")


10 delanteros con mayor puntaje predicho:
A. Griezmann - Puntos: 129.14, Posición: delantero, Precio: 17211813.34 €
A. Budimir - Puntos: 119.98, Posición: delantero, Precio: 16501073.34 €
V. Muriqi - Puntos: 113.52, Posición: delantero, Precio: 16506528.34 €
Sávio - Puntos: 93.72, Posición: delantero, Precio: 15633492.32 €
R. Lewandowski - Puntos: 90.33, Posición: delantero, Precio: 10530381.41 €
G. Moreno - Puntos: 89.41, Posición: delantero, Precio: 12564471.07 €
I. Aspas - Puntos: 85.37, Posición: delantero, Precio: 9634829.11 €
M. Oyarzabal - Puntos: 83.25, Posición: delantero, Precio: 11158284.21 €
J. Strand Larsen - Puntos: 78.64, Posición: delantero, Precio: 10815100.74 €
F. Torres - Puntos: 76.74, Posición: delantero, Precio: 8555802.19 €
Datos de delanteros exportados a sugerencia_delanteros.csv


In [161]:
top_10_defensas = resultados[resultados['Posicion'] == 'defensa'].sort_values(by='Puntos_Predichos', ascending=False).head(10)

print("\n10 defensas con mayor puntaje predicho:")
for i, row in top_10_defensas.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Posición: {row['Posicion']}, Precio: {row['Precio_Predicho']:.2f} €")

top_10_defensas = top_10_defensas.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_defensas.to_csv('sugerencia_defensas.csv', index=False)
print("Datos de defensas exportados a sugerencia_defensas.csv")


10 defensas con mayor puntaje predicho:
J. Cancelo - Puntos: 87.35, Posición: defensa, Precio: 9916051.15 €
F. Lejeune - Puntos: 86.59, Posición: defensa, Precio: 9131181.57 €
Ó. De Marcos - Puntos: 86.26, Posición: defensa, Precio: 9344685.41 €
D. Rico - Puntos: 81.63, Posición: defensa, Precio: 9849615.48 €
A. Witsel - Puntos: 81.28, Posición: defensa, Precio: 9525026.14 €
Y. Berchiche - Puntos: 76.69, Posición: defensa, Precio: 8691783.60 €
M. Mármol - Puntos: 76.57, Posición: defensa, Precio: 8691783.60 €
J. Luis Gayà - Puntos: 75.77, Posición: defensa, Precio: 10015623.69 €
R. Le Normand - Puntos: 68.92, Posición: defensa, Precio: 8442553.34 €
M. Diakhaby - Puntos: 61.63, Posición: defensa, Precio: 7593239.50 €
Datos de defensas exportados a sugerencia_defensas.csv


In [162]:
top_10_mediocentros = resultados[resultados['Posicion'] == 'mediocentro'].sort_values(by='Puntos_Predichos', ascending=False).head(10)

print("\n10 mediocentros con mayor puntaje predicho:")
for i, row in top_10_mediocentros.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Posición: {row['Posicion']}, Precio: {row['Precio_Predicho']:.2f} €")

top_10_mediocentros = top_10_mediocentros.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_mediocentros.to_csv('sugerencia_mediocentros.csv', index=False)
print("Datos de mediocentros exportados a sugerencia_mediocentros.csv")


10 mediocentros con mayor puntaje predicho:
J. Bellingham - Puntos: 127.72, Posición: mediocentro, Precio: 17147016.67 €
N. Williams - Puntos: 122.38, Posición: mediocentro, Precio: 17343248.22 €
B. Zaragoza - Puntos: 89.75, Posición: mediocentro, Precio: 15381661.97 €
F. Valverde - Puntos: 87.35, Posición: mediocentro, Precio: 11533234.23 €
R. De Paul - Puntos: 85.99, Posición: mediocentro, Precio: 9529708.79 €
O. Sancet - Puntos: 85.96, Posición: mediocentro, Precio: 9696092.47 €
J. Guerra - Puntos: 84.44, Posición: mediocentro, Precio: 10156555.00 €
T. Kroos - Puntos: 84.40, Posición: mediocentro, Precio: 11753885.66 €
V. Tsygankov - Puntos: 81.15, Posición: mediocentro, Precio: 9977632.83 €
Y. Herrera - Puntos: 76.88, Posición: mediocentro, Precio: 9728080.96 €
Datos de mediocentros exportados a sugerencia_mediocentros.csv


In [163]:
top_10_porteros = resultados[resultados['Posicion'] == 'portero'].sort_values(by='Puntos_Predichos', ascending=False).head(10)

print("\n10 porteros con mayor puntaje predicho:")
for i, row in top_10_porteros.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Posición: {row['Posicion']},Precio: {row['Precio_Predicho']:.2f} €")

top_10_porteros = top_10_porteros.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_porteros.to_csv('sugerencia_porteros.csv', index=False)
print("Datos de porteros exportados a sugerencia_porteros.csv")


10 porteros con mayor puntaje predicho:
R. Fernández - Puntos: 41.61, Posición: portero,Precio: 1826404.63 €
V. Guaita - Puntos: 38.12, Posición: portero,Precio: 3562612.15 €
A. Ferreira - Puntos: 33.83, Posición: portero,Precio: 1546346.02 €
D. Mariño - Puntos: 30.66, Posición: portero,Precio: 797461.62 €
U. Marrero - Puntos: 5.16, Posición: portero,Precio: 261041.83 €
P. Reina - Puntos: 0.00, Posición: portero,Precio: 204783.12 €
I. Grbić - Puntos: 0.00, Posición: portero,Precio: 191326.02 €
T. Courtois - Puntos: 0.00, Posición: portero,Precio: 1474886.91 €
J. Doménech - Puntos: 0.00, Posición: portero,Precio: 222850.41 €
D. Cárdenas - Puntos: 0.00, Posición: portero,Precio: 305675.81 €
Datos de porteros exportados a sugerencia_porteros.csv
